## 10.3 RNN 循环神经网络

到目前为止,我们所见的神经网络都有一个主要特征,没有记忆的概念.每次输入对于模型而言都是独立的,模型没有保存每个输入之间的状态.这样的网络被称为前馈神经网络,即参数从输入层到输出层单向传播.

前馈神经网络处理时序数据,我们需要一次将整个数据批次送入模型.(否则一个一个输入模型完全没有时序(输入顺序)的概念),这里我们是将 5d 的数据整合成了一个,一次性输入模型.

神经网络模型并不等同于人脑中的神经元及神经元之间的连接(要复杂出几个数量级),但是确实神经网络模型是受实际神经元的启发而来.当人本身读取时序数据时,会有联系上下文的过程,现在在读这本书,输入是一个一个的词,最终大脑会保持上下文的记忆,词语 -> 句子 -> 段落 -> 上下文含义.

循环神经网络 RNN 也是采用了同样的原则,但是是一个极其简化的模型.rnn 不断迭代处理时序数据,并保存一个状态,其中包含了与已处理过的数据相同的信息.从结构上看 rnn 相当于有一个内部的环状结构.

![simplernn](simplernn.png)

在处理两个不同的独立序列的数据之间,rnn 状态会被重置.因此仍然可以将一个序列看着一个单独的数据点,即网络的单个输入.真正改变的是数据点不再是单个步骤中处理,网络内部会对序列元素进行遍历.


为了将序列和循环的概念讲清楚,我们动手实现一个极简的 rnn 网络.

```py
state_t = 0
for input_t in input_sequence:
    output_t = f(input_t, state_t)
    state_t = output_t
```


RNN

- 网络状态初始没有输出,初始化为0
- 每次输出都会将 输入+状态 -> f
- 每次新的输出都会保存到状态,进行下一轮循环.


为了将概念明确化,来一个具体的例子.


In [1]:
import numpy as np

timesteps = 100  #输入数据的时间步长
input_features = 32  #输入数据的特征空间维度
output_features = 64  #输出数据的特征空间维度
inputs = np.random.random((timesteps, input_features))  #输入数据,这里演示所以仅为随机噪声
state_t = np.zeros((output_features, ))  #状态初始化为全 0
W = np.random.random((output_features, input_features))  #随机初始化权重
U = np.random.random((output_features, output_features))  #
b = np.random.random((output_features, ))  #
successive_outputs = []
for input_t in inputs:
    output_t = np.tanh(np.dot(W, input_t) + np.dot(U, state_t) + b)  #计算输出
    successive_outputs.append(output_t)  #保存输出
    state_t = output_t  #更新状态
final_output_sequence = np.concatenate(
    successive_outputs, axis=0)  #拼接,最终输出的形状是(timesteps, output_features)


rnn 就是个for 循环,不断重复利用上一次的迭代结果,仅此而已.

当然 rnn 还有非常多的变种,都满足上面的定义,我们的例子只是最简单的实现.


In [ ]:
output_t = np.tanh(np.dot(W, input_t) + np.dot(U, state_t) + b)

我们例子中的 rnn 按照时间展开

![unrolled_lstm_1](unrolled_lstm_1.png)


## keras 中的循环层

与上面小节示例相同的,keras 中是 `SimpleRNN` 层.

while,只有一点区别.`SimpleRNN` 层就像其他 keras 层一样处理的是批量序列,而不是示例中的单个序列.

`SimpleRNN` 层的输入是 `(batch_size, timesteps, input_features)` 而不是 `(timesteps, input_features)`,


In [10]:
from tensorflow import keras
import tensorflow.keras.layers as layers

In [11]:
num_features = 14
inputs = keras.Input(shape=(None, num_features))
outputs = layers.SimpleRNN(16)(inputs)

我们可以将 `timesteps` 设置为 `None`,这样网络就可以处理任意长度的序列.如果模型需要处理可变长度序列,这个设置会非常受用.

一般情况下,如果可以建议将所有的序列设置为相同的输入长度.这样 `model.summary()` 能够显示输出长度信息,方便调试,并且能够进行一些 rnn 的性能优化(下一节).


keras 的所有 rnn 层(SimpleRNN, LSTM, and GRU) 都可以工作在两者模式:

- 返回每个时间步的连续输出的完整序列,即最终的输出形状是 batch_size, timesteps, output_features)
- 只返回每个序列的最终输出,形状是 (batch_size, output_features)
- 模式由 `return_sequences` 参数控制.默认 `return_sequences =False`.


In [13]:
num_features = 14
steps = 120
inputs = keras.Input(shape=(steps, num_features))
outputs = layers.SimpleRNN(16, return_sequences=False)(inputs)  #False
print(outputs.shape)

(None, 16)


In [14]:
num_features = 14
steps = 120
inputs = keras.Input(shape=(steps, num_features))
outputs = layers.SimpleRNN(16, return_sequences=True)(inputs)  #True
print(outputs.shape)

(None, 120, 16)


In [15]:
inputs = keras.Input(shape=(steps, num_features))
x = layers.SimpleRNN(16, return_sequences=True)(inputs)
x = layers.SimpleRNN(16, return_sequences=True)(x)
outputs = layers.SimpleRNN(16)(x)

有时为了提高模型的表现,将几个递归层堆叠,此时中间层需要返回完整的输出序列,只有结果层只返回最终的结果.


### LSTM

---

这一节很懵,LSTM 原理请参考其他文章

---

实践中很少直接使用 `SimpleRNN`,`SimpleRNN` 是个非常简化的模型,尽管理论上 `SimpleRNN` 可以保留很长一段时间内的输入带来的影响,但是只有一个循环层,这里遇到了类似前馈神经网络的梯度消失问题.随着不断向网络添加 `SimpleRNN` 层,模型最终将无法训练.

keras 还有其他类型的 rnn 层,lstm 和 fru.这两个变种都是为了解决 `SimpleRNN` 遇到的梯度消失问题.

ltsm 增加了一种跨越信息步长的信息传输,假设有一条传送带,运行方向与处理的序列平行,序列中的信息可以被送上传送带,然后被传送到更晚的时间,在需要的时候再被传送回来.这就是 ltsm 的作用: 为以后保存信息,防止旧信息在处理过程中逐渐消失.(有些类似残差连接-出自相同的想法)


![unrolled_lstm_1](unrolled_lstm_1.png)

如上图是一个 `simplernn`,也算是 ltsm 的起点,这里为权重矩阵添加了 o 表示这是计算输出的矩阵,后面还有很多权重矩阵以示区别.


![unrolled_lstm_2](unrolled_lstm_2.png)

我们为这幅图添加了额外的数据流,其可以跨过时间线携带信息.它在不同的时间步叫做 $C_t$,这些信息将会对单元产生影响: 它将与输入连接和循环连接进行运算(通过一个密集变换,即与权重矩阵作点积运算,再加上偏置,最终应用一个激活函数),从而影响下一个时间步的状态(通过一个激活函数和乘法运算).携带数据流是一种条件下一个输出和下一个状态的方法.

到这里还很简单.


![unrolled_lstm_3](unrolled_lstm_3.png)

妙的是携带数据流下一个值的计算方法,涉及 3 个转换,形式上都与 `SimpleRNN` 计算输出相同.

```py
y = activation(dot(state_t, U) + dot(input_t, W) + b)
```

但是这 3 个变换都有各自的权重矩阵,这里用 i f k 表示了.

```py
output_t = activation(dot(state_t, Uo) + dot(input_t, Wo) + dot(C_t, Vo) + bo)
i_t = activation(dot(state_t, Ui) + dot(input_t, Wi) + bi)
f_t = activation(dot(state_t, Uf) + dot(input_t, Wf) + bf)
k_t = activation(dot(state_t, Uk) + dot(input_t, Wk) + bk)
```

下一步可以计算下一个 $C_t$ 即 $C_(t+1)$

```py
c_t+1 = i_t * k_t + c_t * f_t
```


这里可以对 ltsm 的公式进行一些解读,但是解读最终可能没有什么作用,这些公式的意义最终只与权重有关,而最终权重是优化算法决定,而不是人决定的.

---

省略一些对 ltsm 的说明,接本小节最后一段

---

在这里我们不需要了解 ltsm 单元的具体架构,只需要记住 ltsm 单元的作用: 允许过去的信息稍后重新进入,从而解决梯度消失问题.
